## build openai llm model

In [12]:
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

chat_model = ChatOpenAI(model="gpt-3.5-turbo")

## simple test

In [ ]:
from langchain.schema.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="""你是一個資安專家，根據CNS16190消費者物聯網之網宇安全：基準要求事項，回答相關問題"""
    ),
    HumanMessage(content="詳述CNS16190的適用範圍"),
]

response = chat_model.invoke(messages)
response

## test with template

### create a template

In [1]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """你是一個{context}，根據CNS16190消費者物聯網之網宇安全：基準要求事項，回答相關問題"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "資安專家"
question = "簡述CNS16190的適用範圍"

review_prompt_template.format_messages(context=context, question=question)


[SystemMessage(content='你是一個資安專家，根據CNS16190消費者物聯網之網宇安全：基準要求事項，回答相關問題'),
 HumanMessage(content='簡述CNS16190的適用範圍')]

### test inference with openai llm

In [ ]:
review_chain = review_prompt_template | chat_model

In [ ]:
context = "資安專家"
question = "CNS16190可以用在哪裡？"
result = review_chain.invoke({"context": context, "question": question})

In [ ]:
print(f'result.content: {result.content}\n')

print('\nresponse_metadata\n')
for key, value in result.response_metadata.items():
    print(f'{key}: {value}')

print('\n\nusage_metadata\n')
for key, value in result.usage_metadata.items():
    print(f'{key}: {value}')

## retrieve docs

In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain_huggingface import HuggingFaceEmbeddings

with open("../docs/CNS16190-zh_TW_only_provision.md", "r", encoding="utf-8") as f:
    data = f.read()

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

md_splits = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=True
).split_text(data)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=10,
    separators=[
        "\n\n",
        "\n",
        " ",
        "。",
        "，",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)
chunked_documents = text_splitter.split_documents(md_splits)

### embed via openai

In [2]:
reviews_vector_db = Qdrant.from_documents(
    chunked_documents,
    OpenAIEmbeddings(),
    location=":memory:",
)

In [3]:
# ## 控制措施4-1：對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由。

# 表B.1提供以結構化方式記錄此等衡量理由之綱要。此係容許其他利害相關者(例：保證評鑑者、供應鏈成員、安全研究者或零售商)判定是否正確且適切適用控制措施。

# 例1.製造者於其網站上與產品說明一起發布表B.1之完整版本。

# 例2.製造者填寫表B.1以保存內部紀錄。一段時間後，外部保證組織依本標準評鑑產品，並請求提供與產品安全設計相關之資訊。製造者可易於提供此等資訊，因其包含於表B.1中。

# 消費者IoT裝置不適用或未執行控制措施之情況包括：

# - 當裝置係受限制裝置時，某些安全措施之實作，對已識別風險(安全或隱私)係不可能或不適切。
# - 未納入控制措施中所描述之功能性(例：僅提供資料而未要求鑑別的裝置)。

# 例3.電池壽命有限之窗戶感測器於觸發時，經由遠端相關聯服務發送警示，並經由集線器控制。因相較於其他消費者IoT裝置，其電池壽命及處理能力有限，故其為受限制裝置。此外，由於使用者經由集線器控制裝置，使用者無需使用通行碼或其他鑑別機制直接鑑別裝置。

question = """「對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由」是控制措施哪一條？"""

print('similarity\n')
relevant_docs = reviews_vector_db.search(question, search_type='similarity',k=5)

for item in relevant_docs:
    print(f'{item}\n')

print('\n\nsimilarity_score_threshold\n')
relevant_docs = reviews_vector_db.search(question, search_type='similarity_score_threshold',k=5, score_threshold=0.7)

for item in relevant_docs:
    print(f'{item}\n')

print('\n\nmmr\n')
relevant_docs = reviews_vector_db.search(question, search_type='mmr',k=5)

for item in relevant_docs:
    print(f'{item}\n')

similarity

page_content='消費者IoT裝置不適用或未執行控制措施之情況包括：' metadata={'Header 1': '4. 報告實作', 'Header 2': '控制措施4-1：對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由。', '_id': '8f597290b7834eb6a1f3b4a8280f66f7', '_collection_name': '43a45e184a0e4b2cb18eb37b858d7756'}

page_content='。本標準旨在協助消費者IoT裝置製造者' metadata={'Header 1': '6. 消費者IoT裝置之資料保護控制措施', '_id': 'fcf6742cb12241dcb44ef67b08da656d', '_collection_name': '43a45e184a0e4b2cb18eb37b858d7756'}

page_content='本標準中控制措施之實作' metadata={'Header 1': '4. 報告實作', '_id': 'b8363e56ce0b4e8f90866d7f5db7a1e9', '_collection_name': '43a45e184a0e4b2cb18eb37b858d7756'}

page_content='若消費者IoT裝置偵測出對其軟體之未經授權的變更，其將能通知正確之利害相關者' metadata={'Header 1': '5. 消費者IoT裝置之網宇安全控制措施', 'Header 2': '5.7 確保軟體完整性', 'Header 3': '控制措施5.7-2：若偵測出對軟體之未經授權的變更，則裝置宜對使用者及/或管理者發出警示，且不宜連接至比執行警示功能所必要之網路更廣的網路。', '_id': '5334bbff9326465195170c8342c81dd1', '_collection_name': '43a45e184a0e4b2cb18eb37b858d7756'}

page_content='，控制措施的適用性取決於各裝置' metadata={'Header 1': '4. 報告實作', '_id': 'ce3e1b4a8f594fdaa93d62188dad

### embed via sentence-transformer

In [4]:
sentence_transformer_model_root = "../sentence_transformer_model"
sentence_transformer_model = "multi-qa-mpnet-base-dot-v1"

reviews_vector_db = Qdrant.from_documents(
    chunked_documents,
    HuggingFaceEmbeddings(
        model_name=os.path.join(
            sentence_transformer_model_root, sentence_transformer_model
        )
    ),
    location=":memory:",
)


e:\dev\auray\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
question = """「對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由」是控制措施哪一條？"""

print('similarity\n')
relevant_docs = reviews_vector_db.search(question, search_type='similarity',k=5)

for item in relevant_docs:
    print(f'{item}\n')

print('\n\nsimilarity_score_threshold\n')
relevant_docs = reviews_vector_db.search(question, search_type='similarity_score_threshold',k=5, score_threshold=0.7)

for item in relevant_docs:
    print(f'{item}\n')

print('\n\nmnr\n')
relevant_docs = reviews_vector_db.search(question, search_type='mmr',k=5)

for item in relevant_docs:
    print(f'{item}\n')

similarity

page_content='。本標準旨在協助消費者IoT裝置製造者' metadata={'Header 1': '6. 消費者IoT裝置之資料保護控制措施', '_id': 'cbe18beb961d45d4a458afe6e57c01d9', '_collection_name': '49f4becd9f6a4706906bbb34ed8c640c'}

page_content='，製造者不提供裝置未要求之任何背景過程、內核延伸、命令、程式或工具' metadata={'Header 1': '5. 消費者IoT裝置之網宇安全控制措施', 'Header 2': '5.6 最小化暴露之攻擊面', 'Header 3': '控制措施5.6-5：對裝置之預期用途或運作，製造者宜僅啟用其所使用或所要求的軟體服務。', '_id': '72156e01c55d491e9e2c1e486b1c3a40', '_collection_name': '49f4becd9f6a4706906bbb34ed8c640c'}

page_content='。本標準透過使用非必備宜使用控制措施(建議)，提供一定程度之彈性。' metadata={'Header 1': '4. 報告實作', '_id': 'a59f6730a6dd4a8aa9dd9c7727587322', '_collection_name': '49f4becd9f6a4706906bbb34ed8c640c'}

page_content='消費者IoT裝置不適用或未執行控制措施之情況包括：' metadata={'Header 1': '4. 報告實作', 'Header 2': '控制措施4-1：對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由。', '_id': '36af57b7911b4d49827e8b0b92c8e1f0', '_collection_name': '49f4becd9f6a4706906bbb34ed8c640c'}

page_content='。雖於裝置製造者自己之產品中對其修復至關重要' metadata={'Header 1': '5. 消費者IoT裝置之網宇安全控制措施', 'Header 2': '5.2 實作管理脆弱性

## ask llm with rag

In [7]:
from langchain.schema.runnable import RunnablePassthrough

reviews_retriever = reviews_vector_db.as_retriever(k=5)

In [8]:
context = "資安專家"
question = "簡述CNS16190的適用範圍"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content='你是一個資安專家，根據CNS16190消費者物聯網之網宇安全：基準要求事項，回答相關問題'),
 HumanMessage(content='簡述CNS16190的適用範圍')]

### invoke openai llm model

In [15]:
review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
)

### invoke other hf model

In [5]:
import torch
import transformers
from langchain_huggingface import HuggingFacePipeline
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
)

model_name = "MediaTek-Research/Breeze-7B-32k-Instruct-v1_0"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir="../llm_model/", trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
streamer = TextStreamer(tokenizer=tokenizer, skip_prompt=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="../llm_model/",
    device_map="auto",
    low_cpu_mem_usage=True,  # try to limit RAM
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),  # load model in low precision to save memory
    # attn_implementation="flash_attention_2",
)
# Building a LLM QNA chain
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.5,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2048,
    streamer=streamer,
)

huggingFacePipeline = HuggingFacePipeline(pipeline=text_generation_pipeline)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | huggingFacePipeline
)


### ask

In [10]:
question = """「對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由」符合哪一項控制措施？"""

In [11]:
final_result = review_chain.invoke(question)

e:\dev\auray\.venv\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:674: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


</s>


In [13]:
final_result

"System: 你是一個[Document(page_content='消費者IoT裝置不適用或未執行控制措施之情況包括：', metadata={'Header 1': '4. 報告實作', 'Header 2': '控制措施4-1：對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由。', '_id': '87e0a5a46800497b881315e9ca09d80d', '_collection_name': 'd2d95f0566424f66817d6095a9df6faa'}), Document(page_content='。本標準旨在協助消費者IoT裝置製造者', metadata={'Header 1': '6. 消費者IoT裝置之資料保護控制措施', '_id': '007d3a9f8042425094bf439f69abf7a0', '_collection_name': 'd2d95f0566424f66817d6095a9df6faa'}), Document(page_content='本標準中控制措施之實作', metadata={'Header 1': '4. 報告實作', '_id': 'b1a49de6311441cea6487557a2986aaf', '_collection_name': 'd2d95f0566424f66817d6095a9df6faa'}), Document(page_content='。對某些使用案例及下列風險評鑑，可適切應用額外控制措施及本標準內之控制措施。', metadata={'Header 1': '4. 報告實作', '_id': '96628b8335f64af9ba2a6a2855b283ea', '_collection_name': 'd2d95f0566424f66817d6095a9df6faa'})]，根據CNS16190消費者物聯網之網宇安全：基準要求事項，回答相關問題\nHuman: 「對本標準中視為不適用或消費者IoT裝置未滿足之各項建議，應記錄衡量理由」符合哪一項控制措施？"